<a href="https://colab.research.google.com/github/yangr296/ECE539/blob/main/ECE539_Baseline_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
#import data from drive
import tensorflow as tf
import numpy as np
import random
from google.colab import drive
from sklearn.model_selection import train_test_split


drive.mount('/content/drive')
with open('/content/drive/Shared drives/ECE539/parkinsons/parkinsons.data', 'r') as f:
  tmp = np.genfromtxt(f,delimiter=',')
  np.set_printoptions(precision=3,suppress=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
#function defs for ANN
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # The examples are read at random, in no particular order
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        j = tf.constant(indices[i:min(i + batch_size, num_examples)])
        yield tf.gather(features, j), tf.gather(labels, j)

# Optimization algorithm Stochastic Gradient Descent
def sgd(param, grad, lr, batch_size):
  param.assign_sub(lr * grad)

# Loss Function
def cross_entropy(z, t):
  # return tf.keras.losses.CategoricalCrossentropy()(t,z)
  # return  -(1/len(z)) * tf.math.square(tf.reduce_sum(tf.math.log(z)*t))
  return tf.reduce_mean(tf.norm(z - t, axis=1) / tf.math.square(tf.norm(t, axis=1)))

def model(x,w):
  # layer with softmax function
  u = np.hstack((np.ones((x.shape[0],1)), x))@w
  u_exp = tf.math.exp(u)
  z = u_exp/tf.reduce_sum(u_exp,axis=1,keepdims=True)
  # if softmax activation is to be used,
  # z = tf.nn.softmax(u)
  return z

In [63]:
# Data Prep
# One-hot output layer encodding
X, y = tmp[1:, 1:], tmp[1:, -7]
X = np.delete(X, 16, 1)
X = X.astype(float)
labels = np.array(1 * [y[:] == 1, y[:] != 1]).T.astype('f4')

# K: Nsamples, d: featureDimension, N: Nclasses
K,d = X.shape
N = labels.shape[1]

X_train, X_test, y_train, y_test = train_test_split(X, labels, train_size=0.8, shuffle=True, random_state=3, stratify=labels)

# Hyperparameters for ANN
batch_size = 40
lr = 0.03 # learning rate
num_epochs = 1

In [64]:
# Learning
w = tf.Variable(tf.random.normal(shape=(d+1,N)), trainable=True)

for epoch in range(num_epochs):
  for x, y in data_iter(batch_size, X_train, y_train):
    # Feed-forward model
    with tf.GradientTape() as g:
      l = cross_entropy(model(x, w),y)

    # Compute gradient on l with respect to w
    dw = g.gradient(l, w)
    print("dw", dw)
    print("w", w)
    # Update parameters using their gradient
    sgd(w, dw, lr, batch_size)

  # After one epoch, Evaluate resubstitution loss
  train_loss = cross_entropy(model(tf.constant(X_train), w),y_train)
  # print(w)
  print(f'epoch {epoch + 1}, loss {float(tf.reduce_mean(train_loss)):f}')

# Evaluation
Z_test = model(X_test,w)
idx = np.argmax(Z_test,axis=1)
y_pred = np.zeros((idx.size, N))
y_pred[np.arange(idx.size),idx] = 1
Cmat = y_test.T@y_pred

print('Confusion Matrix= \n', Cmat)

dw tf.Tensor(
[[nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]
 [nan nan]], shape=(23, 2), dtype=float32)
w <tf.Variable 'Variable:0' shape=(23, 2) dtype=float32, numpy=
array([[ 0.046,  0.738],
       [ 1.378,  1.779],
       [-0.598, -1.175],
       [-0.806, -0.137],
       [ 1.023, -0.882],
       [ 2.121, -0.778],
       [-0.243, -0.225],
       [-0.736,  0.535],
       [-0.024,  1.315],
       [ 0.118,  0.217],
       [-0.139, -0.81 ],
       [ 0.542,  0.377],
       [-1.107, -0.087],
       [-1.122, -0.082],
       [-1.108,  0.217],
       [ 1.112,  0.082],
       [-0.469, -0.419],
       [ 0.648,  0.625],
       [-0.235,  0.908],
       [-1.604, -0.176],
       [ 0.897, -2.662],
       [ 1.623, -0.223],
       [-0.688, -0.551]], dtype=float32)>
dw tf.Tensor(
[[nan nan]
 [nan nan]
 [nan nan